In [213]:
!pip install psycopg2 --quiet
!pip install sentence_transformers --quiet
!pip install konlpy --quiet

In [214]:
import psycopg2
import pandas as pd
import numpy as np
import itertools
import re

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import MinMaxScaler

In [215]:
book = pd.read_csv('/content/drive/MyDrive/project1/book_table_after.csv')

In [216]:
review = pd.read_csv('/content/drive/MyDrive/project1/review_table_after.csv')

In [217]:
book.drop(columns='Unnamed: 0', inplace=True)
review.drop(columns='Unnamed: 0', inplace=True)

In [218]:
book

,isbn,item_id,category,title,author,publisher,p_date,price,page,avg_score,description,chk,keyword
0,9791191825435,294902251,가정/요리/뷰티,맛있어서 지속 가능한 디디미니 다이어트 레시피,미니 박지우,빅피시,2022-05-23,16920,296,10.0,매년수만개의감량후기가잇따르는고단백요리로수많은다이어터의찬사를받고있는디디미니가올해도어김...,0,"['활용', '요리', '찬사', '디저트', '다이어트']"
1,9791192107608,285827577,가정/요리/뷰티,열두 달 향신료 카레,김민지,세미콜론,2022-06-20,18000,224,0.0,커다란간판없이도사람들이모여드는서울최고의카레성지오직향신료카레하나만을먹기위해웨이팅도불사...,0,"['노란색', '먹기', '열두달', '향신료', '서울']"
2,9791164261840,295216320,가정/요리/뷰티,페이스트리 테이블,박성채,더테이블,2022-05-30,28800,256,10.0,디저트카페창업을위한레시피컨설팅북디저트카페를운영하는데있어가장기본이되는핵심메뉴그리고크림...,0,"['시뮬레이션', '파티', '창업', '메뉴', '디저트']"
3,9791138323284,296057122,가정/요리/뷰티,소워니놀이터의 말랑말랑 스퀴시,조윤성,시대인,2022-07-05,18000,194,0.0,기존의종이놀이를뛰어넘어도안과도안사이에솜을넣어말랑말랑하고폭신폭신한촉감의스퀴시를만들수있...,0,"['손재주', '칭찬', '놀이', '놀이터', '만족감']"
4,9791140700233,296633531,가정/요리/뷰티,맛있는 게 좋아서 양조절 다이어트,한아름,길벗,2022-06-22,16650,228,9.8,매일습관만바꿔도이빠진다고기도탄수화물도디저트도포기할수없는맛있는게너무좋은다이어터들에게딱...,0,"['만천', '습관', '탄수화물', '디저트', '다이어트']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2797,9788952227829,35029230,로맨스소설,미 비포 유,조조 모예스,살림,2013-12-24,13500,536,8.9,영국에서입소문만으로최고의베스트셀러가되고이후출간된독일에서는밀리언셀러로서엄청난판매량을기...,0,"['첫출근', '경제', '남녀', '유머', '독일']"
2798,9788952766441,18593792,로맨스소설,그레이의 50가지 그림자 1,제임스,시공사,2012-08-08,10800,416,6.8,년월미국순문학출판사인랜덤하우스빈티지는그레이의가지그림자가출간석달만에시리즈포함총천백만부...,0,"['판매', '뉴욕타임스', '아마존닷컴', '로맨스소설', '억만장자']"
2799,9791189604110,244280066,로맨스소설,상견니,지엔치펑,리플레이,2020-07-14,14220,432,9.8,년대만최고의화제작드라마소설죽은남자친구왕취안성에대한그리움을떨치지못한황위쉬안은이세계의또...,0,"['의혹', '확신', '영혼', '소설', '그리움']"
2800,9791135475832,290418968,로맨스소설,나쁜 생각이 들어서,요안나,로코코,2022-03-17,12150,560,9.9,요안나장편소설졸지에거액의빚과생면부지의어린이복동생이생긴서희학교선배이자금융그룹의대표서지...,0,"['어린이', '거짓말', '학교', '장편소설', '금융']"


In [219]:
review

,index,isbn,user_id,score,rev_content,category
0,0,9791191825435,gkxmzbsl,10,"이 전에 3,4권도 요리의 ‘요‘자도 알지못하던 저한테 있어서 빛과 같은 희망을 준...",0
1,1,9791191825435,닉닉,10,"이전 책들도 구입했는데, 집밥 먹을때는 거의 항상 디디미니님 레시피 따라하게 되더라...",5
2,2,9791191825435,zin,10,드디어 나왔나요!!!바로 주문합니다!!! 책 잘 보고 다이어트 도전 해볼께요!!! ...,0
3,3,9791191825435,이쁜카라얌,10,요 앞 버젼 사려다가 최신간으로 구매 했어요다이어트를 해도 점점 찌는 나 드디어 굴...,0
4,4,9791191825435,feelingood,10,디디미니 언니 레시피 언제나 믿고 만들어먹어요~ 이번 책엔 또 어떤 맛있는 레시피가...,5
...,...,...,...,...,...,...
16318,17192,9791186561096,gksmfqor12,10,여행을 하면 남는건 사진밖에 없다고..,0
16319,17193,9791186561096,반가워요,10,저도 떠나보고 싶게끔 만드네요 ^^,3
16320,17194,9791186561096,Polnareff,10,20대만의 재기발랄함이 멋지네요,5
16321,17195,9791186561096,0ijimin0i,10,유럽 여행을 해보지는 못했지만 이책으로 인해 유럽에 대해더 자세히 알 수 있어서 너...,5


In [ ]:
# 작가 전처리

In [220]:
'''
drop_book = book[~book['author'].str.contains('지음')]
drop_book = drop_book[~drop_book['author'].str.contains('엮음')]
drop_book = drop_book[~drop_book['author'].str.contains('글')]
drop_book = drop_book[~drop_book['author'].str.contains('그림')]
drop_book = drop_book[~drop_book['author'].str.contains('저자')]
drop_book = drop_book[~drop_book['author'].str.contains('원작')]
book.drop(index=drop_book.index, inplace=True)
book.drop(index=book[book['category']=='전집/중고전집'].index, inplace=True)

def writer(str):
    words = ['엮음', '글', '그림', '저자', '원작']
    for word in words:
        str = str.replace(word, '지음')
    str = str.split('지음')[0].strip()
    str = str.split('외')[0].strip()
    str = re.sub(r'[A-Z]+\.', '', str)
    str = str.replace('.', ', ')
    str = str.replace('(', ', ')
    str = str.replace(')', ', ')
    return str

book['author'] = book['author'].apply(lambda x: writer(x))
book.drop(index=book[book['author']==''].index, inplace=True) # 마지막으로 결측치 제거
'''

"\ndrop_book = book[~book['author'].str.contains('지음')]\ndrop_book = drop_book[~drop_book['author'].str.contains('엮음')]\ndrop_book = drop_book[~drop_book['author'].str.contains('글')]\ndrop_book = drop_book[~drop_book['author'].str.contains('그림')]\ndrop_book = drop_book[~drop_book['author'].str.contains('저자')]\ndrop_book = drop_book[~drop_book['author'].str.contains('원작')]\nbook.drop(index=drop_book.index, inplace=True)\nbook.drop(index=book[book['category']=='전집/중고전집'].index, inplace=True)\n\ndef writer(str):\n    words = ['엮음', '글', '그림', '저자', '원작']\n    for word in words:\n        str = str.replace(word, '지음')\n    str = str.split('지음')[0].strip()\n    str = str.split('외')[0].strip()\n    str = re.sub(r'[A-Z]+\\.', '', str)\n    str = str.replace('.', ', ')\n    str = str.replace('(', ', ')\n    str = str.replace(')', ', ')\n    return str\n\nbook['author'] = book['author'].apply(lambda x: writer(x))\nbook.drop(index=book[book['author']==''].index, inplace=True) # 마지막으로 결측치 제거\n"

In [221]:
# 작가 유사도

In [222]:
author = book['author'].str.split(",", expand=True)

In [223]:
all_author = set()
for c in author.columns:
    distinct_author = author[c].str.lower().str.strip().unique()
    all_author.update(distinct_author)
all_author.remove(None)
all_author.remove('')

In [224]:
item_author_mat = book[['isbn', 'author']].copy()
item_author_mat['author'] = item_author_mat['author'].str.lower().str.strip()

In [225]:
for author in all_author:
    item_author_mat[author] = np.where(item_author_mat['author'].str.contains(author), 1, 0)
item_author_mat = item_author_mat.drop(['author'], axis=1)
item_author_mat = item_author_mat.set_index('isbn')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  


In [226]:
author_mat = cosine_similarity(item_author_mat) # 작가가 같으면 1, 아니면 0

In [227]:
sum(author_mat.sum(axis=0) != 1) # 1744개는 같은 작가가 낸 다른 책이 있음.

1744

In [228]:
# 평균 평점
book['avg_score'].isna().sum() # 손댈 것 없음!ㅎㅎ
book['avg_score'].replace(to_replace=0, value=book['avg_score'].mean(), inplace=True)
scaler = MinMaxScaler()
avg_score = scaler.fit_transform(pd.DataFrame(book['avg_score'])[['avg_score']])
avg_score = avg_score.reshape(-1, )

In [18]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

def keyword_out(doc):

    # okt 객체 생성
    okt = Okt()

    # 태깅 작업 후, 품사가 명사인 단어들을 모두 join 해준다.
    tokenized_doc = okt.pos(doc)
    tokenized_nouns = ' '.join(okt.nouns(doc))

    # 최소 1, 최대 1 길이의 단어를 추출한다.
    n_gram_range = (1, 1)

    count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
    candidates = count.get_feature_names_out()


    doc_embedding = model.encode([doc])
    candidate_embeddings = model.encode(candidates)

    # top_n : 추출할 키워드 갯수
    # nr_candidates : 상위 n개의 키워드를 선택하고 이들 중 가장 유사성이 낮은 5개 선택
    top_n = 5
    nr_candidates = 10

    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    embeddings_vals = [candidate_embeddings[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim
    
    embedding_array = np.empty((0, 768), int)
    for idx in candidate:
       embedding_array = np.append(embedding_array, np.array([embeddings_vals[idx]]), axis=0)

    return np.mean(embedding_array, axis=0)

In [22]:
'''
# book['keyword'] = book['description'].apply(keyword_out)
keyword_embeddings = np.empty((0, 768))
for i in range(len(book)):
    # print(i)
    # keyword_embeddings = np.append(keyword_embeddings, np.array([keyword_out(book['description'][i])]), axis=0)
'''

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [24]:
'''
keyword_embeddings.shape
keyword_sim = cosine_similarity(keyword_embeddings, keyword_embeddings)

keyword_sim_df = pd.DataFrame(keyword_sim)
keyword_sim_df
keyword_sim_df.to_csv('/content/drive/MyDrive/project1/keyword_sim.csv')
'''

,0,1,2,3,4,5,6,7,8,9,...,2792,2793,2794,2795,2796,2797,2798,2799,2800,2801
0,1.000000,0.702968,0.770445,0.803958,0.882862,0.714092,0.798897,0.691549,0.818468,0.667781,...,0.651886,0.631441,0.582155,0.639284,0.563956,0.605041,0.565442,0.642826,0.560337,0.631441
1,0.702968,1.000000,0.585268,0.720218,0.669827,0.698237,0.611379,0.710510,0.717907,0.595698,...,0.629627,0.617329,0.597432,0.607442,0.579685,0.573841,0.542465,0.630784,0.558779,0.617329
2,0.770445,0.585268,1.000000,0.711952,0.704006,0.574259,0.728904,0.631733,0.728078,0.685111,...,0.574850,0.593568,0.540606,0.604789,0.562929,0.597045,0.569427,0.535272,0.578361,0.593568
3,0.803958,0.720218,0.711952,1.000000,0.696428,0.688357,0.785422,0.764791,0.901538,0.721125,...,0.706262,0.678693,0.674870,0.679754,0.624309,0.662965,0.601313,0.756321,0.643112,0.678693
4,0.882862,0.669827,0.704006,0.696428,1.000000,0.663859,0.755273,0.633346,0.717296,0.672040,...,0.604123,0.627060,0.617728,0.612333,0.571365,0.591931,0.603065,0.630284,0.634654,0.627060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2797,0.605041,0.573841,0.597045,0.662965,0.591931,0.634275,0.654766,0.656929,0.685340,0.599735,...,0.654386,0.679094,0.614493,0.570263,0.612126,1.000000,0.676073,0.615077,0.615713,0.679094
2798,0.565442,0.542465,0.569427,0.601313,0.603065,0.505981,0.570680,0.650038,0.625468,0.654127,...,0.711788,0.733702,0.673867,0.670970,0.762376,0.676073,1.000000,0.660960,0.636636,0.733702
2799,0.642826,0.630784,0.535272,0.756321,0.630284,0.579960,0.677604,0.668398,0.797451,0.659365,...,0.713594,0.755226,0.832376,0.711466,0.808589,0.615077,0.660960,1.000000,0.687487,0.755226
2800,0.560337,0.558779,0.578361,0.643112,0.634654,0.470719,0.621154,0.625699,0.686290,0.721047,...,0.615842,0.794992,0.678771,0.674614,0.662818,0.615713,0.636636,0.687487,1.000000,0.794992


In [230]:
keyword_mat = pd.read_csv('/content/drive/MyDrive/project1/keyword_sim.csv')
keyword_mat = np.array(keyword_mat.drop(columns='Unnamed: 0'))

In [231]:
# 테스트
score = pd.DataFrame(keyword_mat[200], columns=['score'], index=book.index)
book[book.index.isin(score['score'].nlargest(5).index)]

,isbn,item_id,category,title,author,publisher,p_date,price,page,avg_score,description,chk,keyword
188,9791130689876,292824466,경제경영,나는 대출 없이 0원으로 소형 아파트를 산다,"잭파시, 최경천,",다산북스,2022-04-19,18000,412,9.900000,직장인년차통장에는만원밖에없었다부동산경매에입찰하기위한보증금만원이없어서대출을받아야했다그...,0,"['대출', '억원', '투자자', '비용', '부동산경매']"
200,9788957822074,295631436,경제경영,경매대마왕 반드시 부자 되는 투자의 소신,심태승,국일증권경제연구소,2022-05-31,15300,304,9.800000,구독자만유튜버경매대마왕대한민국부동산경매위굿프렌드심태승부원장의투자노하우가완벽히공개됐다...,0,"['노하우', '부자', '부동산경매', '투자자', '경매']"
210,9791162542705,290848034,경제경영,따라 하면 무조건 돈 버는 실전 부동산 경매,"유근용, 정민우",비즈니스북스,2022-03-22,19800,440,10.000000,년부동산경매분야화제의베스트셀러왕초보도바로돈버는부동산경매의기술의두저자가실전경매사례를중...,0,"['베스트셀러', '부동산', '경매', '수익률', '투자자']"
225,9788957825945,276636094,경제경영,전설로 떠나는 월가의 영웅,"피터 린치, 존 로스차일드",국일증권경제연구소,2021-07-30,23400,464,9.700000,어떤기업이든공부하지않고주식을사면카드를보지않고포커게임에임하는것과같다사람들이부동산에서돈...,0,"['수익', '쇼핑', '게임', '부동산', '투자자']"
516,9791130690568,295482581,달력/기타,[큰글자도서] 나는 대출 없이 0원으로 소형 아파트를 산다,"잭파시, 최경천,",다산북스,2022-05-16,45000,412,5.210064,직장인년차통장에는만원밖에없었다부동산경매에입찰하기위한보증금만원이없어서대출을받아야했다그...,0,"['대출', '억원', '투자자', '비용', '부동산경매']"


In [ ]:
# 키워드 자카드 유사도

In [ ]:
'''
def JaccardSimilarity(i, j):
    mom = set(i).union(set(j))
    son = set(i).intersection(set(j))
    return len(son)/len(mom)

keyword_sim = []
for i in range(len(book['keyword'])):
    ls = []
    for j in range(len(book['keyword'])):
        jaccard_sim = JaccardSimilarity(book['keyword'][i], book['keyword'][j])
        ls.append(jaccard_sim)
    keyword_sim.append(ls)
book['keyword_sim'] = keyword_sim
'''

In [ ]:
# 카테고리 유사도

In [232]:
count_vect = CountVectorizer(min_df=0, ngram_range=(1, 1))
category_vect = count_vect.fit_transform(book['category'])
category_mat = cosine_similarity(category_vect, category_vect)

In [233]:
# 감정 유사도

In [234]:
emotion = review[['isbn', 'category']].groupby(['isbn', 'category']).size().unstack()
emotions = pd.DataFrame(emotion.idxmax(axis=1), columns=['emotion'])

In [235]:
book = book.set_index('isbn', drop=True)
book = book.join(emotions)

In [236]:
def emotion_sim(emotion): # emotion은 0-5까지 숫자로 변환된 감정
    review.groupby('isbn').count().max(level=0)
    emotion_sim = np.where(book['emotion']==emotion, 1, 0)
    return emotion_sim

In [241]:
book # index를 isbn으로 하고 emotion column을 추가한 새로운 book table

,item_id,category,title,author,publisher,p_date,price,page,avg_score,description,chk,keyword,emotion
isbn,,,,,,,,,,,,,
9791191825435,294902251,가정/요리/뷰티,맛있어서 지속 가능한 디디미니 다이어트 레시피,미니 박지우,빅피시,2022-05-23,16920,296,10.000000,매년수만개의감량후기가잇따르는고단백요리로수많은다이어터의찬사를받고있는디디미니가올해도어김...,0,"['활용', '요리', '찬사', '디저트', '다이어트']",5.0
9791192107608,285827577,가정/요리/뷰티,열두 달 향신료 카레,김민지,세미콜론,2022-06-20,18000,224,5.210064,커다란간판없이도사람들이모여드는서울최고의카레성지오직향신료카레하나만을먹기위해웨이팅도불사...,0,"['노란색', '먹기', '열두달', '향신료', '서울']",1.0
9791164261840,295216320,가정/요리/뷰티,페이스트리 테이블,박성채,더테이블,2022-05-30,28800,256,10.000000,디저트카페창업을위한레시피컨설팅북디저트카페를운영하는데있어가장기본이되는핵심메뉴그리고크림...,0,"['시뮬레이션', '파티', '창업', '메뉴', '디저트']",5.0
9791138323284,296057122,가정/요리/뷰티,소워니놀이터의 말랑말랑 스퀴시,조윤성,시대인,2022-07-05,18000,194,5.210064,기존의종이놀이를뛰어넘어도안과도안사이에솜을넣어말랑말랑하고폭신폭신한촉감의스퀴시를만들수있...,0,"['손재주', '칭찬', '놀이', '놀이터', '만족감']",NaN
9791140700233,296633531,가정/요리/뷰티,맛있는 게 좋아서 양조절 다이어트,한아름,길벗,2022-06-22,16650,228,9.800000,매일습관만바꿔도이빠진다고기도탄수화물도디저트도포기할수없는맛있는게너무좋은다이어터들에게딱...,0,"['만천', '습관', '탄수화물', '디저트', '다이어트']",0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9788952227829,35029230,로맨스소설,미 비포 유,조조 모예스,살림,2013-12-24,13500,536,8.900000,영국에서입소문만으로최고의베스트셀러가되고이후출간된독일에서는밀리언셀러로서엄청난판매량을기...,0,"['첫출근', '경제', '남녀', '유머', '독일']",3.0
9788952766441,18593792,로맨스소설,그레이의 50가지 그림자 1,제임스,시공사,2012-08-08,10800,416,6.800000,년월미국순문학출판사인랜덤하우스빈티지는그레이의가지그림자가출간석달만에시리즈포함총천백만부...,0,"['판매', '뉴욕타임스', '아마존닷컴', '로맨스소설', '억만장자']",0.0
9791189604110,244280066,로맨스소설,상견니,지엔치펑,리플레이,2020-07-14,14220,432,9.800000,년대만최고의화제작드라마소설죽은남자친구왕취안성에대한그리움을떨치지못한황위쉬안은이세계의또...,0,"['의혹', '확신', '영혼', '소설', '그리움']",5.0


In [238]:
# 추천시스템
def res(category_mat, author_mat, keyword_mat, emotion_sim, avg_score,
        index, emotion,
        w1=.1, w2=.1, w3=.35, w4=.1, top_n=5):
    
    category_sim = category_mat[index]
    author_sim = author_mat[index]
    keyword_sim = keyword_mat[index]
    emotion_sim = emotion_sim(emotion)
    
    w5 = 1 - (w1+w2+w3+w4)

    score = pd.Series(w1*category_sim + w2*author_sim + w3*keyword_sim + w4*emotion_sim + w5*avg_score)
    score.index = book.index
    top_pick = score.nlargest(top_n+1).index[1:]

    return top_pick

In [239]:
# top_pick으로 나온 isbn index 사용해서 책 정보 불러오기
top_pick = res(category_mat, author_mat, keyword_mat, emotion_sim, avg_score, index=0, emotion=0)
book.loc[top_pick]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().
  This is separate from the ipykernel package so we can avoid doing imports until


,item_id,category,title,author,publisher,p_date,price,page,avg_score,description,chk,keyword,emotion
isbn,,,,,,,,,,,,,
9791197438707,272563613,가정/요리/뷰티,디디미니의 초간단 인생맛 고단백 저탄수화물 다이어트 레시피,미니 박지우,빅피시,2021-06-07,16650,264,9.9,매년다이어트요리책의새로운역사를쓰는디디미니가더욱더새롭고알찬레시피를가지고돌아왔다언제나자...,0,"['또밀', '파스타', '다이어트', '디저트', '조리법']",5.0
9791197035234,241362504,가정/요리/뷰티,맛있게 살 빠지는 고단백 저탄수화물 다이어트 레시피,미니 박지우,비에이블,2020-05-28,15750,276,9.9,만명이선택한최강다이어터이며베스트셀러고단백저탄수화물다이어트레시피시리즈저자미니박지우의최...,0,"['성공', '취향', '조리법', '다이어트', '음식']",5.0
9791140700233,296633531,가정/요리/뷰티,맛있는 게 좋아서 양조절 다이어트,한아름,길벗,2022-06-22,16650,228,9.8,매일습관만바꿔도이빠진다고기도탄수화물도디저트도포기할수없는맛있는게너무좋은다이어터들에게딱...,0,"['만천', '습관', '탄수화물', '디저트', '다이어트']",0.0
9791165215088,266819159,가정/요리/뷰티,한 달에 7kg 빠지는 다이어트 레시피,송혜영,길벗,2021-03-22,14220,224,9.7,도시락부터주말특식까지다이어트식단걱정은이제그만다이어트레시피관련누적조회수만회현재도매주다...,0,"['활용', '영상', '요리', '샌드위치', '다이어트식단']",0.0
9791155814017,280048698,가정/요리/뷰티,킨포크 가든,존 번스,윌북,2021-10-01,29700,352,10.0,단순하고소박한삶을지향하는라이프스타일매거진킨포크의가든에세이자연의기쁨을삶에들이는방법을담...,0,"['가이드', '기쁨', '라이프스타일', '꽃다발', '채소']",0.0
